# Préparation du dataset Titanic pour le Machine Learning

On veut entraîner un modèle de prédiction de survie sur le dataset Titanic. 
Pour cela, on a besoin de passer par plusieurs étapes

In [42]:
import pandas as pd

titanic = pd.read_csv("Data/titanic-survival.csv")

In [43]:
titanic.head(3)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


## Choix des variables de prédiction en fonction de notre variable d'intérêt

Regardez si vous trouvez des corrélations entre les différentes variables du dataset et le fait de survivre au nauffrage.
Proposez en conséquence Les variables que vous souhaiteriez utiliser

In [44]:
titanic.groupby("survived")["fare"].mean()

survived
0.0    23.353831
1.0    49.361184
Name: fare, dtype: float64

In [45]:
titanic.groupby("survived").mean(numeric_only=True)

,pclass,age,sibsp,parch,fare,body
survived,,,,,,
0.0,2.500618,30.545369,0.521632,0.328801,23.353831,160.809917
1.0,1.962000,28.918228,0.462000,0.476000,49.361184,NaN


In [46]:
titanic.groupby("sex")["survived"].mean()

sex
female    0.727468
male      0.190985
Name: survived, dtype: float64

In [47]:
titanic.groupby("pclass")["survived"].mean()

pclass
1.0    0.619195
2.0    0.429603
3.0    0.255289
Name: survived, dtype: float64

In [48]:
titanic.groupby("embarked")["survived"].mean()

embarked
C    0.555556
Q    0.357724
S    0.332604
Name: survived, dtype: float64

## Passage des variables de mon modèle au bon type  
Les variables catégorielles ne sont pas utilisables en l'état pour entraîner un modèle de Machine Learning, il faut les transformer en un ensemble de variables indicatrices.
Renseignez vous sur la méthode `pandas.get_dummies()` et appliquez là aux variables concernées.  
Appliquez le paramètre de la fonction `drop_first = True`. À quoi sert-il?

In [49]:
help(pd.get_dummies)

Help on function get_dummies in module pandas.core.reshape.reshape:

get_dummies(data, prefix=None, prefix_sep='_', dummy_na: 'bool' = False, columns=None, sparse: 'bool' = False, drop_first: 'bool' = False, dtype: 'Dtype | None' = None) -> 'DataFrame'
    Convert categorical variable into dummy/indicator variables.
    
    Parameters
    ----------
    data : array-like, Series, or DataFrame
        Data of which to get dummy indicators.
    prefix : str, list of str, or dict of str, default None
        String to append DataFrame column names.
        Pass a list with length equal to the number of columns
        when calling get_dummies on a DataFrame. Alternatively, `prefix`
        can be a dictionary mapping column names to prefixes.
    prefix_sep : str, default '_'
        If appending prefix, separator/delimiter to use. Or pass a
        list or dictionary as with `prefix`.
    dummy_na : bool, default False
        Add a column to indicate NaNs, if False NaNs are ignored.
  

In [76]:
import numpy as np
liste_var = ["age", "fare", "sibsp", "parch", 
            "sex_male", "pclass_2.0", "pclass_3.0"]

titanic_ml = pd.get_dummies(titanic, columns=["sex","pclass"], 
                            drop_first = True, dtype=np.uint8)
titanic_ml = titanic_ml.dropna(subset=liste_var)
titanic_ml.shape

(1045, 15)

In [77]:
titanic_ml.columns

Index(['survived', 'name', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin',
       'embarked', 'boat', 'body', 'home.dest', 'sex_male', 'pclass_2.0',
       'pclass_3.0'],
      dtype='object')

In [78]:

titanic_varexp=titanic_ml.loc[:, liste_var]

In [79]:
titanic_varexp = titanic_varexp
titanic_varexp.shape

(1045, 7)

## Standardisation des données  
Renseignez-vous sur ce que sont la standardisation/la normalisation des données. 
À quoi servent ces méthodes?
Utilisez le package `scikit-learn` pour appliquer une standardisation à vos données numériques.

In [80]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(titanic_varexp)

In [81]:
X.shape

(1045, 7)

In [82]:
Y=titanic_ml.loc[:, "survived"]
Y.shape

(1045,)

## Début de l'entraînement de mon modèle

### Train/Test Split

In [83]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y,
                                               test_size=0.2)

print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

(836, 7) (209, 7) (836,) (209,)


## Initialisation et entraînement du modèle

On utilise l'agorithme d'arbre de décision

In [84]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()

decision_tree.fit(Xtrain, Ytrain)

DecisionTreeClassifier()

In [88]:
y_predict_test=decision_tree.predict(Xtest)
print(Xtest.shape, y_predict_test.shape, Ytest.shape)

(209, 7) (209,) (209,)


In [89]:
from sklearn.metrics import accuracy_score  

accuracy_score(y_predict_test, Ytest)

0.7033492822966507